# Amostragem

## Carregamento da base de dados

In [3]:
# Biblioteca para trabalhar com base de dados
import pandas as pd
# Acesso a funções randômicas
import random
# Extrair média, cálculos de somatório, contagem, etc
import numpy as np

In [4]:
# Variável para armazenar os dados do arquivo census
dataset = pd.read_csv('credit_data.csv')

In [5]:
# Comando .shape para identificar quantas linhas e colunas possui a base de dados dataset
dataset.shape

(2000, 5)

In [6]:
# Comando .head() para trazer o cabeçalho da tabela, trazendo os cinco primeiros registros
dataset.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [7]:
# Comando .tail() para trazer o os cinco últimos registros da tabela
dataset.tail()

,i#clientid,income,age,loan,c#default
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0
1999,2000,69436.579552,56.152617,7378.833599,0


## Amostragem aleatória simples

In [11]:
# Função para retornar uma quantidade de amostras simples solicitadas, que será a mesma amostra. Essa função pode ser utilizada por qualquer dataframe 
def amostragem_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras, random_state=1)

In [12]:
# Variável que vai receber a função, que invoca o dataframe dataset e retorna a amostra simples com a quantidade fixa de 100 itens do dataframe
df_amostra_aleatoria_simples = amostragem_aleatoria_simples(dataset, 1000)

# Comando .shape para identificar quantas linhas e colunas possui o dataframe armazenado na variável df_amostra_aleatoria_simples, confirmando a quantidade de 100 registros
df_amostra_aleatoria_simples.shape

(1000, 5)

## Amostragem sistemática

In [38]:
# Função para determinar todo o cálculo da amostragem sistemática
def amostragem_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras # Armazenar na variável 'intervalo' o cálculo do resultado da divisão do dataframe e quantidade de amostras escolhidas
  random.seed(1) # Definir que o número aleatório escolhido não mude (caso isso aconteça o resultado final também seria diferente a cada compilação)
  inicio = random.randint(0, intervalo) # Armazenar número escolhido entre 0 e o intervalo calculado
  indices = np.arange(inicio, len(dataset), step = intervalo) # Armazenar o array determinado pela função do numpy 'np.arrange'
  amostra_sistematica = dataset.iloc[indices] # Criação de um novo dataframe com os valores calculados anteriormente através da função .iloc 
  return amostra_sistematica # Retorno do novo dataframe

In [37]:
df_amostra_sistematica = amostragem_sistematica(dataset, 1000) # Chamar a função com os parâmetros dataset e 100 elementos da amostra
df_amostra_sistematica.shape # Verificar quantidade de linhas e colunas da amostra

(1000, 6)

## Amostragem por grupos

In [17]:
# Função que vai ser passado todos os grupos e retorna o dataframe citado anteriormente
def amostragem_agrupamento(dataset, numero_grupos): #
  intervalo = len(dataset) / numero_grupos # Armazenar na variável 'intervalo' o cálculo do resultado da divisão do dataframe pelo número de grupos informados no parâmetro, esse cálculo serve para indicar quantos elementos no máximo terá cada grupo

  grupos = [] # Lista vazia
  id_grupo = 0 # Variável que armazena o id dos grupos, começando do grupo 0
  contagem = 0 # Variável que conta até 3256, que é quando ocorre uma mudança de grupo
  
  # Percorrer todas as linhas do dataset e atribuir os grupo para cada um dos registros
  for _ in dataset.iterrows(): # Função iterrows() percorre cada registro do dataset
    grupos.append(id_grupo) # Adicionar na lista cada id_grupo
    contagem += 1 # Iterando a contagem
    if contagem > intervalo: # Se a contagem > intervalo
      contagem = 0 # Zera o valor contagem
      id_grupo += 1 # Adiciona um valor no id_grupo

  
  dataset['grupo'] = grupos # Acessar o atributo 'grupo' que vai receber a variável grupos, como a variável já existe no dataframe o comando fará a atualização com base na variável 'grupos'
  random.seed(1) # Garantir que o número aleatório escolhido não varie a cada execução
  grupo_selecionado = random.randint(0, numero_grupos) # Selecionar o grupo
  return dataset[dataset['grupo'] == grupo_selecionado] # Retorna os valores dentro do dataframe

In [18]:
df_amostra_agrupamento = amostragem_agrupamento(dataset, 2) # Chamar a função amostragem_agrupamento com o parâmetro do dataframe e o valor de 325
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts() # Quantidade de linhas e colunas do agrupamento, acesso ao número do grupo selecionado e sua quantidade de linhas

((1001, 6), 0    1001
 Name: grupo, dtype: int64)

## Amostra estratificada

In [19]:
from sklearn.model_selection import StratifiedShuffleSplit # Classe 'StratifiedShuffleSplit' que faz a seleção das amostras para fazer a amostragem estratificada

In [22]:
dataset['c#default'].value_counts()

0    1717
1     283
Name: c#default, dtype: int64

In [25]:
# Função para calular a amostragem estratificada
def amostragem_estratificada(dataset, percentual, campo):
  split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
  for _, y in split.split(dataset, dataset[campo]):
    df_y = dataset.iloc[y]
  return df_y

In [26]:
df_amostra_estratificada = amostragem_estratificada(dataset, 0.5,'c#default') # Chamando a função e passando os parâmetros de dataframe e porcentagem do dataframe solicitada para a amostra.
df_amostra_estratificada.shape

(1000, 6)

## Amostragem de reservatório

In [27]:
# Função 
def amostragem_reservatorio(dataset, amostras):
  stream = []
  for i in range(len(dataset)): # Laço de repetição para percorrer o dataset
    stream.append(i) # Adiciona cada elemento no dataset

  i = 0 # Variável i recebe zero
  tamanho = len(dataset) # Variável tamanho recebe a quantidade de registro do dataset

  reservatorio = [0] * amostras # Armazena os índices dos elementos que forem sorteados
  for i in range(amostras): # Laço de repetição para percorrer a amostra
    reservatorio[i] = stream[i] # Inicializa a contagem e armazena os elementos

  while i < tamanho: # Laço de repetição para fazer a seleção dos valores
    j = random.randrange(i + 1) # Função randrange()
    if j < amostras: # Condicional para definir todos os valores sorteados
      reservatorio[j] = stream[i]
    i += 1

  return dataset.iloc[reservatorio] # Dentro de dataset haverá os ids sorteados e através da função .iloc[] serão selecionados os registros de acordo com os ids selecionados.
  

In [28]:
df_amostragem_reservatorio = amostragem_reservatorio(dataset, 1000) # Chama a função com o dataset e a quantidade de amostras.
df_amostragem_reservatorio.shape

(1000, 6)

## Comparativo dos resultados

In [102]:
# Cálculo da média
mean = dataset['age'].mean(), dataset['income'].mean(), dataset['loan'].mean()
a = list(mean)

def resultado(lista):
  result = [a-lista for a,lista in zip(a,lista)] 
  return result

In [118]:
# Cálculo da média para amostra aleatória simples
meanAleatoriaSimples = df_amostra_aleatoria_simples['age'].mean(), df_amostra_aleatoria_simples['income'].mean(), df_amostra_aleatoria_simples['loan'].mean() 
b = list(meanAleatoriaSimples)
listaB = resultado(b)
print(abs(sum(listaB)))





236.43380796264273


In [119]:
# Cálculo da média para amostra sistemática
meanAleatoriaSistematica = df_amostra_sistematica['age'].mean(), df_amostra_sistematica['income'].mean(), df_amostra_sistematica['loan'].mean() 
c = list(meanAleatoriaSistematica)
listaC = resultado(c)
print(abs(sum(listaC)))


422.4206290472668


In [121]:
# Cálculo da média para amostragem por grupos
meanAmostragemGrupos = df_amostra_agrupamento['age'].mean(), df_amostra_agrupamento['income'].mean(), df_amostra_agrupamento['loan'].mean() 
d = list(meanAmostragemGrupos)
listaD = resultado(d)
print(abs(sum(listaD)))

538.8232951423156


In [123]:
# Cálculo da média para amostra estratificada
meanAmostraEstratificada = df_amostra_estratificada['age'].mean(), df_amostra_estratificada['income'].mean(), df_amostra_estratificada['loan'].mean() 
e = list(meanAmostraEstratificada)
listaE = resultado(e)
print(abs(sum(listaE)))

251.1322334266477


In [125]:
# Cálculo da média para amostragem de reservatório
meanAmostraReservatorio = df_amostragem_reservatorio['age'].mean(), df_amostragem_reservatorio['income'].mean(), df_amostragem_reservatorio['loan'].mean() 
f = list(meanAmostraReservatorio)
listaF = resultado(f)
print(abs(sum(listaF)))

235.3759660621078


**Considerando o cálculo de todas as médias em relação a primeira média, pode-se concluir que a média para amostragem de reservatório foi a que melhor se aproximou do resultado da média. Foi utilizada a base de dados 'credit_data.csv'.**